In [1]:
!pip install requests lxml tqdm

In [2]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from utils import *

In [3]:
with open("./config.json", "r") as fp:
    config = json.load(fp)

In [4]:
DATA_DIR_PATH = config["data_dir_path"]

# Read SNLI datasets and generate UD tree and concat to df

In [10]:
df_snli_train = pd.read_csv(os.path.join(DATA_DIR_PATH, "snli_train.tsv"), delimiter='\t', index_col=0)
df_snli_dev = pd.read_csv(os.path.join(DATA_DIR_PATH, "snli_dev.tsv"), delimiter='\t', index_col=0)
df_snli_test = pd.read_csv(os.path.join(DATA_DIR_PATH, "snli_test.tsv"), delimiter='\t', index_col=0)

In [11]:
def concat_udtree_to_df(df):
    udtree1_list = []
    udtree2_list = []
    
    sample_list = list(df.itertuples())
    for i, sample in enumerate(tqdm(sample_list)):
        udtree1 = ET.tostring(sentence2tree(sample.tokens1), encoding="utf-8").decode()
        udtree2 = ET.tostring(sentence2tree(sample.tokens2), encoding="utf-8").decode()
        udtree1_list.append(udtree1)
        udtree2_list.append(udtree2)

    df["udtree1"] = udtree1_list
    df["udtree2"] = udtree2_list

In [12]:
concat_udtree_to_df(df_snli_train)
df_snli_train.head()

  0%|          | 0/549361 [00:00<?, ?it/s]

AttributeError: 'Pandas' object has no attribute 'tokens1'

In [41]:
concat_udtree_to_df(df_snli_dev)
df_snli_dev.head()

  0%|          | 0/9842 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
concat_udtree_to_df(df_snli_test)
df_snli_test.head()

In [ ]:
df_snli_train.to_csv(os.path.join(DATA_DIR_PATH, "snli_train.tsv"), sep="\t")
df_snli_dev.to_csv(os.path.join(DATA_DIR_PATH, "snli_dev.tsv"), sep="\t")
df_snli_test.to_csv(os.path.join(DATA_DIR_PATH, "snli_test.tsv"), sep="\t")